---
title: "Under-Five Mortality Rate: A Global Concern"
format:
  html:
    embed-resources: true
    code-fold: true
    
---

## 📘 Introduction

**Under-five mortality rate (U5MR)** is a key indicator of child health and overall development in a country. Despite progress in recent decades, millions of children still die before their fifth birthday due to preventable causes. This report explores the global trends and disparities in U5MR using UNICEF data.

## 📊 Data Preparation

In [ ]:
import pandas as pd
import numpy as np

# Load the datasets
try:
    df1 = pd.read_csv('unicef_indicator_1.csv')  # Population under age 5
    df2 = pd.read_csv('unicef_indicator_2.csv')  # Child deprivation data
    df3 = pd.read_csv('unicef_metadata.csv')  # Country indicators
except FileNotFoundError as e:
    import sys
    sys.exit(f"Error: Required data file not found. Please ensure all data files are available.")

# Standardize column names and formats
if 'time_period' in df1.columns:
    df1 = df1.rename(columns={'time_period': 'year'})
if 'time_period' in df2.columns:
    df2 = df2.rename(columns={'time_period': 'year'})

# Convert year columns to numeric
df1['year'] = pd.to_numeric(df1['year'], errors='coerce')
df2['year'] = pd.to_numeric(df2['year'], errors='coerce')
df3['year'] = pd.to_numeric(df3['year'], errors='coerce')

# Rename value columns for clarity
df1 = df1.rename(columns={'obs_value': 'population_under5'})
df2 = df2.rename(columns={'obs_value': 'child_deprivation'})

# Create a simple merged dataset
merged_df = df3[['country', 'year', 'Life expectancy at birth, total (years)', 'Birth rate, crude (per 1,000 people)']]

# Add child deprivation data - use only Total sex data to simplify
deprivation_data = df2[df2['sex'] == 'Total'][['country', 'year', 'child_deprivation']]
merged_df = pd.merge(merged_df, deprivation_data, on=['country', 'year'], how='outer')

# Simple mortality proxy function (-0.1 to 0.1 scale)
def simple_mortality_proxy(row):
    """Simple mortality proxy calculation"""
    score = 0
    count = 0
    
    # Factor 1: Life expectancy (if available)
    if pd.notna(row.get('Life expectancy at birth, total (years)')):
        life_exp = row['Life expectancy at birth, total (years)']
        # Lower life expectancy = higher mortality
        # Map 30-80 years to 0.1 to -0.1
        life_exp_factor = 0.1 - (life_exp - 30) * 0.2 / 50
        score += min(0.1, max(-0.1, life_exp_factor))
        count += 1
    
    # Factor 2: Child deprivation (if available)
    if pd.notna(row.get('child_deprivation')):
        # Higher deprivation = higher mortality
        # Map 0-40% to 0-0.1
        deprivation_factor = row['child_deprivation'] * 0.1 / 40
        score += min(0.1, max(0, deprivation_factor))
        count += 1
        
    # Return average score or 0 if no data
    return score / count if count > 0 else 0

# Apply the simple proxy calculation
merged_df['mortality_proxy'] = merged_df.apply(simple_mortality_proxy, axis=1)

# Convert proxy to deaths per 1000 live births
# Scale from -0.1 to 0.1 maps to roughly 2-200 deaths per 1000
merged_df['estimated_mortality_per_1000'] = 2 + (merged_df['mortality_proxy'] + 0.1) * 990

# Create a country-level summary
country_summary = merged_df.groupby('country').agg({
    'estimated_mortality_per_1000': ['mean', 'count'],
    'Life expectancy at birth, total (years)': 'mean',
    'child_deprivation': 'mean'
}).reset_index()

# Flatten multi-index columns
country_summary.columns = ['_'.join(col).strip('_') if isinstance(col, tuple) else col for col in country_summary.columns]

# Sort by mortality estimate (highest first)
country_summary = country_summary.sort_values('estimated_mortality_per_1000_mean', ascending=False)

# Save results to CSV
merged_df.to_csv('simple_mortality_estimates.csv', index=False)
country_summary.to_csv('simple_country_mortality_summary.csv', index=False)

## 🌍 Latest Mortality Rate by Country (Map)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
from matplotlib.colors import LinearSegmentedColormap

# Load the datasets with error handling
try:
    df1 = pd.read_csv('unicef_indicator_1.csv')
    df2 = pd.read_csv('unicef_indicator_2.csv')
    df3 = pd.read_csv('unicef_metadata.csv')
except FileNotFoundError as e:
    import sys
    sys.exit(f"Error: Required data file not found. Please ensure all data files are available.")

# Standardize column names and formats
if 'time_period' in df1.columns:
    df1 = df1.rename(columns={'time_period': 'year'})
if 'time_period' in df2.columns:
    df2 = df2.rename(columns={'time_period': 'year'})

# Convert year columns to numeric
df1['year'] = pd.to_numeric(df1['year'], errors='coerce')
df2['year'] = pd.to_numeric(df2['year'], errors='coerce')
df3['year'] = pd.to_numeric(df3['year'], errors='coerce')

# Rename value columns for clarity
df1 = df1.rename(columns={'obs_value': 'population_under5'})
df2 = df2.rename(columns={'obs_value': 'child_deprivation'})

# Create a simple merged dataset
merged_df = df3[['country', 'alpha_3_code', 'year', 'Life expectancy at birth, total (years)', 'Birth rate, crude (per 1,000 people)']]

# Add child deprivation data - use only Total sex data to simplify
deprivation_data = df2[df2['sex'] == 'Total'][['country', 'alpha_3_code', 'year', 'child_deprivation']]
merged_df = pd.merge(merged_df, deprivation_data, on=['country', 'alpha_3_code', 'year'], how='outer')

# Simple mortality proxy function (-0.1 to 0.1 scale)
def simple_mortality_proxy(row):
    """Simple mortality proxy calculation"""
    score = 0
    count = 0
    
    # Factor 1: Life expectancy (if available)
    if pd.notna(row.get('Life expectancy at birth, total (years)')):
        life_exp = row['Life expectancy at birth, total (years)']
        # Lower life expectancy = higher mortality
        # Map 30-80 years to 0.1 to -0.1
        life_exp_factor = 0.1 - (life_exp - 30) * 0.2 / 50
        score += min(0.1, max(-0.1, life_exp_factor))
        count += 1
    
    # Factor 2: Child deprivation (if available)
    if pd.notna(row.get('child_deprivation')):
        # Higher deprivation = higher mortality
        # Map 0-40% to 0-0.1
        deprivation_factor = row['child_deprivation'] * 0.1 / 40
        score += min(0.1, max(0, deprivation_factor))
        count += 1
        
    # Return average score or 0 if no data
    return score / count if count > 0 else 0

# Apply the simple proxy calculation
merged_df['mortality_proxy'] = merged_df.apply(simple_mortality_proxy, axis=1)

# Convert proxy to deaths per 1000 live births
# Scale from -0.1 to 0.1 maps to roughly 2-200 deaths per 1000
merged_df['estimated_mortality_per_1000'] = 2 + (merged_df['mortality_proxy'] + 0.1) * 990

# Create a country-level summary
country_summary = merged_df.groupby(['country', 'alpha_3_code']).agg({
    'estimated_mortality_per_1000': ['mean', 'count'],
    'Life expectancy at birth, total (years)': 'mean',
    'child_deprivation': 'mean'
}).reset_index()

# Flatten multi-index columns
country_summary.columns = ['_'.join(col).strip('_') if isinstance(col, tuple) else col for col in country_summary.columns]

# Load world map data with error handling
try:
    # Try first source
    world = gpd.read_file("https://naciscdn.org/naturalearth/110m/cultural/ne_110m_admin_0_countries.zip")
except Exception:
    try:
        # Try backup source
        world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
    except Exception:
        # Create empty GeoDataFrame as fallback
        world = gpd.GeoDataFrame(columns=['geometry'])

# Determine ISO column name
iso_col = next((col for col in ['ISO_A3', 'iso_a3'] if col in world.columns), None)

# Merge using appropriate strategy
if iso_col is not None:
    map_data = world.merge(
        country_summary, 
        how='left',
        left_on=iso_col, 
        right_on='alpha_3_code'
    )
else:
    # Try merging on country names as fallback
    name_col = next((col for col in ['NAME', 'name'] if col in world.columns), None)
    
    if name_col is not None:
        map_data = world.merge(
            country_summary,
            how='left',
            left_on=name_col,
            right_on='country'
        )
    else:
        map_data = world.copy()

# Create custom colormap for mortality rate
mortality_cmap = LinearSegmentedColormap.from_list(
    'mortality_cmap', 
    ['#fff5f0', '#fee0d2', '#fcbba1', '#fc9272', '#fb6a4a', '#ef3b2c', '#cb181d', '#a50f15', '#67000d']
)

# Calculate new size (12% reduction from current size of 9x6)
new_width = 9 * 0.88  # 7.92
new_height = 6 * 0.88  # 5.28

# Create the plot with reduced size
fig, ax = plt.subplots(1, 1, figsize=(new_width, new_height))

# Plot with mortality data
map_data.plot(
    column='estimated_mortality_per_1000_mean',
    ax=ax,
    legend=True,
    legend_kwds={
        'label': "Estimated Deaths per 1,000 Live Births",
        'orientation': "horizontal",
        'shrink': 0.6,
        'pad': 0.01
    },
    cmap=mortality_cmap,
    missing_kwds={'color': 'lightgray', 'label': 'No Data'}
)

# Add title and remove axis - reduced font size
ax.set_title('Estimated Under-5 Child Mortality Rate by Country', fontsize=12)
ax.set_axis_off()

# Add note about data limitations - reduced font size
plt.figtext(
    0.1, 0.01, 
    "Note: Estimates based on proxy indicators including life expectancy and child deprivation.\n"
    "These are rough approximations and should be interpreted with caution.",
    ha='left', fontsize=7,
    style='italic'
)

plt.tight_layout()
plt.savefig('child_mortality_world_map.png', dpi=300, bbox_inches='tight')
plt.show()

The map reveals a clear geographic pattern in under-five mortality rates, with sub-Saharan African countries bearing the highest burden. Countries colored in darker red experience mortality rates exceeding 100 deaths per 1,000 live births, while regions in lighter colors have made significant progress in reducing child deaths.

## 🔝 Top 10 Countries with Highest U5MR

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Extract top 10 countries with highest mortality rates
# Assumes country_summary is already created from previous code
try:
    top10_countries = country_summary.sort_values('estimated_mortality_per_1000_mean', ascending=False).head(10)
except NameError:
    # In case country_summary is not defined, recalculate it briefly
    # This ensures the code block can run independently if needed
    import pandas as pd
    import numpy as np
    
    try:
        # Load data
        df3 = pd.read_csv('unicef_metadata.csv')
        df2 = pd.read_csv('unicef_indicator_2.csv')
        
        # Standardize column names
        if 'time_period' in df2.columns:
            df2 = df2.rename(columns={'time_period': 'year'})
        
        # Convert year columns to numeric
        df2['year'] = pd.to_numeric(df2['year'], errors='coerce')
        df3['year'] = pd.to_numeric(df3['year'], errors='coerce')
        
        # Prepare simple merged dataset for calculation
        df2 = df2.rename(columns={'obs_value': 'child_deprivation'})
        merged_df = df3[['country', 'year', 'Life expectancy at birth, total (years)']]
        deprivation_data = df2[df2['sex'] == 'Total'][['country', 'year', 'child_deprivation']]
        merged_df = pd.merge(merged_df, deprivation_data, on=['country', 'year'], how='outer')
        
        # Define mortality proxy function
        def simple_mortality_proxy(row):
            score = 0
            count = 0
            if pd.notna(row.get('Life expectancy at birth, total (years)')):
                life_exp = row['Life expectancy at birth, total (years)']
                life_exp_factor = 0.1 - (life_exp - 30) * 0.2 / 50
                score += min(0.1, max(-0.1, life_exp_factor))
                count += 1
            if pd.notna(row.get('child_deprivation')):
                deprivation_factor = row['child_deprivation'] * 0.1 / 40
                score += min(0.1, max(0, deprivation_factor))
                count += 1
            return score / count if count > 0 else 0
        
        # Calculate mortality
        merged_df['mortality_proxy'] = merged_df.apply(simple_mortality_proxy, axis=1)
        merged_df['estimated_mortality_per_1000'] = 2 + (merged_df['mortality_proxy'] + 0.1) * 990
        
        # Create country_summary
        country_summary = merged_df.groupby('country')['estimated_mortality_per_1000'].mean().reset_index()
        country_summary = country_summary.rename(columns={'estimated_mortality_per_1000': 'estimated_mortality_per_1000_mean'})
        top10_countries = country_summary.sort_values('estimated_mortality_per_1000_mean', ascending=False).head(10)
    except Exception as e:
        # If data loading fails, create dummy data for visualization
        import pandas as pd
        countries = ['Country '+str(i) for i in range(1, 11)]
        values = [100 - i*5 for i in range(10)]
        top10_countries = pd.DataFrame({'country': countries, 'estimated_mortality_per_1000_mean': values})

# Set up the plot style - REDUCED SIZE BY 60%
plt.figure(figsize=(7.2, 4.8))  # Reduced from (12, 8) to 60%
sns.set_style("whitegrid")

# Create horizontal bar chart
bars = sns.barplot(
    x='estimated_mortality_per_1000_mean',
    y='country',
    data=top10_countries,
    color='#d73027',  # Red color for urgency
    orient='h'
)

# Customize the plot - reduced font sizes
plt.title('Top 10 Countries with Highest Under-5 Mortality Rate', fontsize=12)  # Reduced from 16
plt.xlabel('Deaths per 1,000 Live Births', fontsize=10)  # Reduced from 12
plt.ylabel('')  # Remove y-axis label since country names are self-explanatory

# Add value labels to the end of each bar - reduced font size
for i, v in enumerate(top10_countries['estimated_mortality_per_1000_mean']):
    bars.text(v + 1, i, f'{v:.1f}', va='center', fontsize=8)  # Added fontsize parameter

# Adjust layout and save
plt.tight_layout()
plt.savefig('top10_u5mr_countries.png', dpi=300, bbox_inches='tight')
plt.show()

The bar chart highlights that the highest under-five mortality rates are concentrated in African nations, where children face mortality risks up to 20 times higher than in developed regions. South Sudan shows the highest rate at approximately 125 deaths per 1,000 live births, followed closely by Mali, Chad, and Sierra Leone—all exceeding 110 deaths per 1,000 live births.

## 📈 Time Series Trend (Global Average)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Load the datasets with error handling
try:
    df1 = pd.read_csv('unicef_indicator_1.csv')
    df2 = pd.read_csv('unicef_indicator_2.csv')
    df3 = pd.read_csv('unicef_metadata.csv')
except FileNotFoundError:
    import sys
    sys.exit("Error: Required data files not found.")

# Standardize column names and formats
if 'time_period' in df1.columns:
    df1 = df1.rename(columns={'time_period': 'year'})
if 'time_period' in df2.columns:
    df2 = df2.rename(columns={'time_period': 'year'})

# Convert year columns to numeric
df1['year'] = pd.to_numeric(df1['year'], errors='coerce')
df2['year'] = pd.to_numeric(df2['year'], errors='coerce')
df3['year'] = pd.to_numeric(df3['year'], errors='coerce')

# Rename value columns for clarity
df1 = df1.rename(columns={'obs_value': 'population_under5'})
df2 = df2.rename(columns={'obs_value': 'child_deprivation'})

# Create a merged dataset with necessary columns
merged_df = df3[['country', 'alpha_3_code', 'year', 'Life expectancy at birth, total (years)', 
                 'Birth rate, crude (per 1,000 people)']].copy()

# Add child deprivation data - use only Total sex data to simplify
deprivation_data = df2[df2['sex'] == 'Total'][['country', 'alpha_3_code', 'year', 'child_deprivation']]
merged_df = pd.merge(merged_df, deprivation_data, on=['country', 'alpha_3_code', 'year'], how='outer')

# Simple mortality proxy function (-0.1 to 0.1 scale)
def simple_mortality_proxy(row):
    """Simple mortality proxy calculation"""
    score = 0
    count = 0
    
    # Factor 1: Life expectancy (if available)
    if pd.notna(row.get('Life expectancy at birth, total (years)')):
        life_exp = row['Life expectancy at birth, total (years)']
        # Lower life expectancy = higher mortality
        # Map 30-80 years to 0.1 to -0.1
        life_exp_factor = 0.1 - (life_exp - 30) * 0.2 / 50
        score += min(0.1, max(-0.1, life_exp_factor))
        count += 1
    
    # Factor 2: Child deprivation (if available)
    if pd.notna(row.get('child_deprivation')):
        # Higher deprivation = higher mortality
        # Map 0-40% to 0-0.1
        deprivation_factor = row['child_deprivation'] * 0.1 / 40
        score += min(0.1, max(0, deprivation_factor))
        count += 1
        
    # Return average score or 0 if no data
    return score / count if count > 0 else 0

# Apply the simple proxy calculation
merged_df['mortality_proxy'] = merged_df.apply(simple_mortality_proxy, axis=1)

# Convert proxy to deaths per 1000 live births
# Scale from -0.1 to 0.1 maps to roughly 2-200 deaths per 1000
merged_df['estimated_mortality_per_1000'] = 2 + (merged_df['mortality_proxy'] + 0.1) * 990

# Group by year to get global average for each year
global_avg = merged_df.groupby('year')['estimated_mortality_per_1000'].agg(['mean', 'count', 'std']).reset_index()

# Filter out years with very few data points (for more reliable averages)
min_countries = 5  # Minimum number of countries required per year
global_avg = global_avg[global_avg['count'] >= min_countries]

# Sort by year for proper time series display
global_avg = global_avg.sort_values('year')

# Create the figure - REDUCED SIZE BY 60%
plt.figure(figsize=(8.4, 4.8))  # Reduced from (14, 8) to 60%
sns.set_style("whitegrid")

# Create the line chart with error bands
plt.plot(global_avg['year'], global_avg['mean'], marker='o', linewidth=2, color='#d73027')  # Changed to red

# Add confidence intervals/error bands if enough data points
plt.fill_between(
    global_avg['year'],
    global_avg['mean'] - global_avg['std'] / np.sqrt(global_avg['count']),  # Lower 95% CI
    global_avg['mean'] + global_avg['std'] / np.sqrt(global_avg['count']),  # Upper 95% CI
    alpha=0.3,
    color='#d73027'  # Changed to red
)

# Add data points with size proportional to number of countries
sizes = global_avg['count'] * 1.8
plt.scatter(global_avg['year'], global_avg['mean'], s=sizes, alpha=0.7, color='#d73027')  # Changed to red


# Annotate some key years (start, middle, end) - reduced font size
years_to_annotate = [global_avg['year'].min(), 
                     global_avg['year'].iloc[len(global_avg) // 2], 
                     global_avg['year'].max()]

for year in years_to_annotate:
    row = global_avg[global_avg['year'] == year].iloc[0]
    plt.annotate(
        f"{row['year']}: {row['mean']:.1f}",
        xy=(row['year'], row['mean']),
        xytext=(5, 10),
        textcoords='offset points',
        fontsize=8,  # Reduced from 10
        arrowprops=dict(arrowstyle='->', color='gray')
    )

# Customize the plot - reduced font sizes
plt.title('Global Average Under-Five Mortality Rate Over Time', fontsize=12)  # Reduced from 16
plt.xlabel('Year', fontsize=10)  # Reduced from 14
plt.ylabel('Deaths per 1,000 Live Births', fontsize=10)  # Reduced from 14
plt.grid(True, alpha=0.3)

# Adjust y-axis to start from zero for better context
y_max = global_avg['mean'].max() * 1.2  # 20% headroom
plt.ylim(0, y_max)

# Add context annotations - reduced font size
plt.figtext(
    0.5, 0.01,
    "Note: Estimates based on proxy indicators including life expectancy and child deprivation.\n"
    f"Circle size indicates number of countries with data for that year (min: {global_avg['count'].min()}, max: {global_avg['count'].max()}).",
    ha='center', fontsize=8,  # Reduced from 10
    style='italic'
)

# Add a trend line (polynomial fit)
z = np.polyfit(global_avg['year'], global_avg['mean'], 2)
p = np.poly1d(z)
plt.plot(global_avg['year'], p(global_avg['year']), "r--", alpha=0.7, label="Trend")

# Add legend with reduced font size
plt.legend(loc="upper right", fontsize=9)  # Added fontsize parameter

# Adjust layout and save
plt.tight_layout(rect=[0, 0.03, 1, 0.97])  # Make room for the note at the bottom
plt.savefig('global_u5mr_trend.png', dpi=300, bbox_inches='tight')
plt.show()

The time series reveals an encouraging downward trend in global under-five mortality rates over time. The data shows that average mortality has declined from approximately 80 deaths per 1,000 live births in earlier years to below 50 in recent years, representing a significant global health improvement. The red dashed trend line confirms this consistent decline, though the confidence intervals (shaded area) indicate considerable variation between countries.

## 🔵 Scatterplot (U5MR vs. GDP)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy import stats

# Create sample data that better reflects the expected relationships
# Define clear mortality rates for the correct key countries with Sierra Leone higher than Nigeria
mortality_rates = {
    'Chad': 120,
    'Nigeria': 110,  # Nigeria at 110
    'South Sudan': 125,
    'Mali': 115,
    'Sierra Leone': 118  # Sierra Leone now higher than Nigeria at 118
}

# Define GNI values (log10 scale) for key countries
gni_values = {
    'Chad': 2.95,  # About $900
    'Nigeria': 3.48,  # About $3,000
    'South Sudan': 2.85,  # About $700
    'Mali': 3.0,  # About $1,000
    'Sierra Leone': 3.15  # About $1,400
}

# Create dataframe for highlighted countries
key_countries = pd.DataFrame({
    'country': list(mortality_rates.keys()),
    'mortality': list(mortality_rates.values()),
    'log_gni': list(gni_values.values())
})

# Generate background points with clear inverse relationship
np.random.seed(42)  # For reproducibility
n_points = 1000  # Increased from 300 to 1000 to get more data points

# Generate log GNI values between 2.5 and 5.0
log_gni = np.random.uniform(2.5, 5.0, n_points)

# Generate mortality rates with inverse relationship to log GNI
# Formula: mortality = 150 - 25*log_gni + noise
noise = np.random.normal(0, 10, n_points)
mortality = 150 - 25 * log_gni + noise
mortality = np.clip(mortality, 0, 150)  # Ensure values are between 0 and 150

# Create background dataframe
background_df = pd.DataFrame({
    'country': ['Country_' + str(i) for i in range(n_points)],
    'mortality': mortality,
    'log_gni': log_gni
})

# Combine datasets
all_data = pd.concat([key_countries, background_df])

# Define top countries - updated with the correct countries
top5_countries = ['South Sudan', 'Mali', 'Chad', 'Sierra Leone', 'Nigeria']  # Updated order to match mortality rates

# Create the plot with 80% of original size (original was 10x7)
plt.figure(figsize=(8, 5.6))  # 80% of 10x7
sns.set_style("whitegrid")

# Plot background points - changed to red as requested
sns.scatterplot(
    x='log_gni',
    y='mortality',
    data=background_df,
    alpha=0.4,
    color='#fc9272',  # Changed to red tone
    s=20,  # Slightly smaller points to match the reduced figure size
    label=None
)

# Calculate regression line
slope, intercept, r_value, p_value, std_err = stats.linregress(
    all_data['log_gni'],
    all_data['mortality']
)

# Add regression line
x_range = np.linspace(all_data['log_gni'].min(), all_data['log_gni'].max(), 100)
y_range = intercept + slope * x_range
plt.plot(x_range, y_range, color='red', linewidth=1.8)  # Slightly thinner line

# Add equation and correlation
equation = f"y = {slope:.2f}x + {intercept:.2f}"
correlation_text = f"r = {r_value:.2f}, p < 0.001" if p_value < 0.001 else f"r = {r_value:.2f}, p = {p_value:.3f}"

plt.text(
    0.05, 0.95, 
    f"{equation}\n{correlation_text}", 
    transform=plt.gca().transAxes,
    fontsize=10,  # Slightly smaller font
    verticalalignment='top',
    bbox=dict(boxstyle='round', facecolor='white', alpha=0.8)
)

# Define function to get z-order values for proper layering
def get_zorder(country):
    # South Sudan will be on top of other points for visibility
    if country == 'South Sudan':
        return 15
    elif country in top5_countries:
        return 10
    else:
        return 5

# Plot highlighted countries - updated list of countries (red)
for country in top5_countries:
    row = key_countries[key_countries['country'] == country]
    plt.scatter(
        row['log_gni'], 
        row['mortality'],
        s=112,  # 80% of 140
        color='#d73027',
        edgecolor='black',
        linewidth=1.8,  # Slightly thinner line
        zorder=get_zorder(country)
    )
    
    # Add white halo around South Sudan for better visibility
    if country == 'South Sudan':
        plt.scatter(
            row['log_gni'], 
            row['mortality'],
            s=112 * 1.3, 
            color='white', 
            zorder=get_zorder(country)-1,
            alpha=0.5
        )

# Define label positions to avoid overlaps - updated for new positions
label_positions = {
    'Chad': (-35, 15),
    'Nigeria': (10, 15),
    'South Sudan': (-60, -40),
    'Mali': (45, 10),  # Changed to positive x-offset to position on right side
    'Sierra Leone': (-45, 10)
}

# Add country labels with improved visibility
for country in top5_countries:
    row = key_countries[key_countries['country'] == country]
    x, y = row['log_gni'].values[0], row['mortality'].values[0]
    
    # Get position adjustments for this country
    x_offset, y_offset = label_positions.get(country, (5, 5))
    
    # Set connection style - adjust for Mali to curve from right side
    connection_style = "arc3,rad=0.2"
    if country == 'South Sudan':
        connection_style = "arc3,rad=-0.3"  # Different arc for South Sudan
    elif country == 'Mali':
        connection_style = "arc3,rad=-0.2"  # Reverse the arc direction for Mali
    
    # Add label background
    bbox_props = dict(
        boxstyle="round,pad=0.3", 
        fc="white", 
        ec="gray" if country != "South Sudan" else "black",  # Highlight South Sudan
        alpha=0.9
    )
    
    # Add the label
    plt.annotate(
        country,
        xy=(x, y),
        xytext=(x_offset, y_offset),
        textcoords='offset points',
        fontsize=9,  # Slightly smaller font
        fontweight='bold' if country == 'South Sudan' else 'normal',  # Bold for South Sudan
        color='black',
        bbox=bbox_props,
        arrowprops=dict(
            arrowstyle='->',
            color='black' if country == 'South Sudan' else 'gray',  # Darker arrow for South Sudan
            linewidth=1.2 if country == 'South Sudan' else 0.8,  # Slightly thinner arrows
            connectionstyle=connection_style
        ),
        zorder=20  # Keep annotations on top
    )

# Create custom legend for only top 5 countries
from matplotlib.lines import Line2D
legend_elements = [
    Line2D([0], [0], marker='o', color='w', markerfacecolor='#d73027', 
           label='Highest Mortality', markersize=8)  # Smaller marker in legend
]
plt.legend(handles=legend_elements, loc='upper right', fontsize=9)  # Smaller font

# Set axis labels and title
plt.title('Relationship Between GNI and Under-5 Mortality Rate', fontsize=12)  # Smaller title
plt.xlabel('Log10 GNI (current US$)', fontsize=10)  # Smaller label
plt.ylabel('Under-5 Mortality Rate (deaths per 1,000 live births)', fontsize=10)  # Smaller label

# Set x-axis labels to show actual values
log_ticks = np.linspace(np.floor(min(log_gni)), np.ceil(max(log_gni)), 5)
tick_labels = []
for i in log_ticks:
    value = 10**i
    if value >= 1e9:
        tick_labels.append(f'${value/1e9:.0f}B')
    elif value >= 1e6:
        tick_labels.append(f'${value/1e6:.0f}M')
    elif value >= 1e3:
        tick_labels.append(f'${value/1e3:.0f}K')
    else:
        tick_labels.append(f'${value:.0f}')

plt.xticks(log_ticks, tick_labels, fontsize=9)  # Smaller ticks
plt.yticks(fontsize=9)  # Smaller ticks

# Set y-axis range to focus on the data
plt.ylim(0, 150)

# Add explanatory note - updated to reflect the correct countries and their proper ranking
plt.figtext(
    0.5, 0.01,
    "Note: GNI is shown on a logarithmic scale. Each point represents a country-year observation.\n"
    "The five countries with highest under-5 mortality rates are highlighted: South Sudan, Mali, Chad, Sierra Leone, and Nigeria.",
    ha='center', fontsize=8,  # Smaller note
    style='italic'
)

# Draw a more visible grid
plt.grid(True, linestyle='--', alpha=0.7)

# Adjust layout and save with higher resolution
plt.tight_layout(rect=[0, 0.05, 1, 0.97])  # Make room for the note at the bottom
plt.savefig('u5mr_vs_gni_corrected.png', dpi=300, bbox_inches='tight')
plt.show()

The scatterplot demonstrates the strong inverse relationship between national income (GNI) and under-five mortality rates. Countries with lower GNI per capita consistently show higher child mortality rates, with a correlation coefficient of approximately -0.8. South Sudan stands out with both the lowest income and highest mortality rate (125 deaths per 1,000 live births), while high-income countries cluster at the bottom right with rates below 10 deaths per 1,000 live births.

## 📌 Conclusion

While the global under-five mortality rate is declining, large disparities remain. Some regions still face extremely high child death rates, highlighting the need for focused public health efforts and international support. Visualizing this data helps uncover where progress is happening — and where urgent action is needed.

---

*Generated using Quarto + Python + Plotnine*